In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import torch as tc
genders = ['female', 'male']
male = ['male']
female = ['female']

In [2]:
their_semantics = [
    'Subject',
    'Measuring station',
    'repetition no.',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Back neck point to waist',
    'Height (m)',
]
our_semantic = [
    'subject',
    'measures_station',
    'repetition',
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'neck_waist',
    'stature', # 5.1.1
]

station_list = ["MOVE4D", "Expert", "Our"]

measures = pd.read_pickle(f'data/cleaned_measures.zip')
measures = measures[their_semantics]
measures.columns = our_semantic
our_measures = pd.read_pickle("data/our_measures.zip")
measures = pd.concat([our_measures, measures]).drop(columns=["gender", "stature"])
measures = measures[measures['measures_station'].isin(["MOVE4D", "Expert measurer 1", "our"])]
measures.replace("Expert measurer 1", "Expert", inplace=True)
measures.replace("our", "Our", inplace=True)

In [3]:
measures

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,neck_waist,repetition,measures_station,subject
0,890.102844,750.670166,1049.499756,628.432495,292.741119,325.192902,418.211090,1,Our,IEEEP2_01
1,895.189270,753.261353,1051.522095,625.345581,294.438354,326.743317,426.129272,2,Our,IEEEP2_01
2,903.245544,682.914734,947.787170,538.580383,274.551392,314.705322,417.136688,1,Our,IEEEP2_03
3,886.251953,664.944397,951.866211,537.981140,274.649323,316.283325,414.106720,2,Our,IEEEP2_03
4,1098.261108,981.130737,1141.172607,673.190918,343.884399,369.490021,431.907532,1,Our,IEEEP2_07
...,...,...,...,...,...,...,...,...,...,...
1992,1190.000000,1120.000000,1360.000000,740.000000,380.000000,375.000000,405.000000,2,Expert,IEEEP2_71
1997,931.730000,864.896000,988.994000,572.149000,319.426000,370.596000,442.053000,1,MOVE4D,IEEEP2_72
1998,922.700000,863.313000,993.492000,572.987000,320.638000,369.121000,439.612000,2,MOVE4D,IEEEP2_72
2007,930.000000,825.000000,985.000000,565.000000,305.000000,355.000000,435.000000,1,Expert,IEEEP2_72


# CALC ICC

In [4]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure)['ICC'].loc[2]
        measures_of_station.append(icc)
    all_measures.append(measures_of_station)
our_icc = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [5]:
a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.973184,73.583120,71,72,6.212652e-48,"[0.96, 0.98]"
1,ICC2,Single random raters,0.973191,75.032571,71,71,1.198188e-47,"[0.96, 0.98]"
2,ICC3,Single fixed raters,0.973695,75.032571,71,71,1.198188e-47,"[0.96, 0.98]"
3,ICC1k,Average raters absolute,0.986410,73.583120,71,72,6.212652e-48,"[0.98, 0.99]"
4,ICC2k,Average random raters,0.986413,75.032571,71,71,1.198188e-47,"[0.98, 0.99]"
5,ICC3k,Average fixed raters,0.986672,75.032571,71,71,1.198188e-47,"[0.98, 0.99]"


In [6]:
our_icc.index = ['Expert', "MOVE4D", 'Our']
our_icc

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,neck_waist
Expert,0.984964,0.995512,0.993420,0.985788,0.899936,0.980211,0.908226
MOVE4D,0.994299,0.997657,0.999204,0.998120,0.995792,0.995974,0.973715
Our,0.996868,0.998141,0.998908,0.998231,0.994649,0.995912,0.973695


# CALC SEM

In [7]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')['ICC'].loc[2]
        sem = a[measure].std() * np.sqrt(1-icc)
        measures_of_station.append(sem)
    all_measures.append(measures_of_station)
our_sem = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [8]:
our_sem.index = ['Expert', "MOVE4D", 'Our']
our_sem.T.round(4)

,Expert,MOVE4D,Our
bust_chest_girth,12.3293,7.9139,5.6561
waist_girth,8.5865,6.4644,5.8106
hip_girth,7.9206,2.7550,3.0873
thigh_girth,6.9722,2.5773,2.6495
upper_arm_girth,12.6139,2.4972,2.5938
neck_girth,4.8045,2.3456,2.3739
neck_waist,9.4215,4.8049,3.7130


# CALC PSD

In [9]:
def calculate_psd(measure):
    data = measures
    f1 = data['repetition'] == 1
    f2 = data['repetition'] == 2
    data = data[ f1 | f2 ][['subject', 'measures_station', 'repetition', measure]]
    data = data.pivot_table(values=measure, index=['subject', 'repetition'], columns="measures_station")
    data = data.dropna()
    

    column_wise = []
    for column1 in data.columns:
        row_wise = []
        for column2 in data.columns:
            if column1 == column2:
                row_wise.append(np.nan)
            else:
                d1 = data[column1]
                d2 = data[column2]
                var = (d1-d2).var()
                sem1 = our_sem[measure].loc[column1]**2
                sem2 = our_sem[measure].loc[column2]**2
                value = np.sqrt(var + (sem1/2) + (sem2/2))
                row_wise.append(value)
        column_wise.append(row_wise)
    data = pd.DataFrame(column_wise, index=data.columns, columns=data.columns)
    data.index.name = None
    return data

In [10]:
neck_result = calculate_psd(measure='neck_girth')
MX = pd.Series({"MOVE/Expert": neck_result.loc["MOVE4D"]["Expert"]})
neck_result = neck_result.loc['Our'][:-1].append(MX)

In [11]:
bust_result = calculate_psd(measure='bust_chest_girth')
MX = pd.Series({"MOVE/Expert": bust_result.loc["MOVE4D"]["Expert"]})
bust_result = bust_result.loc['Our'][:-1].append(MX)

In [12]:
waist_result = calculate_psd(measure='waist_girth')
MX = pd.Series({"MOVE/Expert": waist_result.loc["MOVE4D"]["Expert"]})
waist_result = waist_result.loc['Our'][:-1].append(MX)

In [13]:
hip_result = calculate_psd(measure='hip_girth')
MX = pd.Series({"MOVE/Expert": hip_result.loc["MOVE4D"]["Expert"]})
hip_result = hip_result.loc['Our'][:-1].append(MX)

In [14]:
arm_result = calculate_psd(measure='upper_arm_girth')
MX = pd.Series({"MOVE/Expert": arm_result.loc["MOVE4D"]["Expert"]})
arm_result = arm_result.loc['Our'][:-1].append(MX)

In [15]:
thigh_result = calculate_psd(measure='thigh_girth')
MX = pd.Series({"MOVE/Expert": thigh_result.loc["MOVE4D"]["Expert"]})
thigh_result = thigh_result.loc['Our'][:-1].append(MX)

In [16]:
neck_waist_result = calculate_psd(measure='neck_waist')
MX = pd.Series({"MOVE/Expert": neck_waist_result.loc["MOVE4D"]["Expert"]})
neck_waist_result = neck_waist_result.loc['Our'][:-1].append(MX)

In [17]:
def calculate_mad():
    all_measures = []
    for station in station_list:
        measures_of_station = []
        a = measures[measures['measures_station'] == station]
        f1 = a['repetition'] == 1
        f2 = a['repetition'] == 2
        for measure in our_semantic[3:-1]:
            mean = np.nanmean(abs(a[f1][measure].to_numpy() - a[f2][measure].to_numpy()))
            measures_of_station.append(mean)
        all_measures.append(measures_of_station)
    return pd.DataFrame(all_measures, columns=our_semantic[3:-1], index=['MOVE4D','Expert','Our']).T

In [18]:
styles = [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "125%"),
                              ("color", 'black'),
                              ("font-weight", '700')])]

In [19]:
mad_table = calculate_mad()
mad_table = mad_table.style.set_caption("OUR vs MOVE4D MAD statistic result").set_table_styles(styles)
mad_table.format(precision=4)


,MOVE4D,Expert,Our
bust_chest_girth,8.5292,12.5347,6.8866
waist_girth,6.4874,9.1319,5.6992
hip_girth,3.0823,8.8194,3.5551
thigh_girth,2.6278,7.8472,2.9425
upper_arm_girth,2.7248,8.2014,2.6521
neck_girth,2.3442,5.2431,2.6996
neck_waist,5.1012,10.4514,4.2303


In [20]:
sem_table = our_sem.T.round(4)
sem_table = sem_table.style.set_caption("OUR vs MOVE4D SEM statistic result").set_table_styles(styles)
sem_table.format(precision=4)

,Expert,MOVE4D,Our
bust_chest_girth,12.3293,7.9139,5.6561
waist_girth,8.5865,6.4644,5.8106
hip_girth,7.9206,2.7550,3.0873
thigh_girth,6.9722,2.5773,2.6495
upper_arm_girth,12.6139,2.4972,2.5938
neck_girth,4.8045,2.3456,2.3739
neck_waist,9.4215,4.8049,3.7130


In [21]:
data_psd = pd.DataFrame(
    [neck_result, bust_result, waist_result, hip_result, arm_result,thigh_result, neck_waist_result],
    index=our_semantic[3:-1]).round(2)
data_psd.columns = ['Our vs Expert', "Our vs MOVE4D", 'MOVE vs Expert']
data_psd.to_csv("output/our_psd.csv")
data_psd = data_psd.style.set_caption("OUR vs MOVE4D PSD statistic result").set_table_styles(styles)
data_psd.format(precision=2)

,Our vs Expert,Our vs MOVE4D,MOVE vs Expert
bust_chest_girth,12.03,7.44,10.17
waist_girth,22.12,20.73,22.43
hip_girth,23.05,13.26,22.21
thigh_girth,15.03,11.94,14.78
upper_arm_girth,16.62,6.02,15.73
neck_girth,14.58,10.43,11.48
neck_waist,19.98,13.70,20.22
